In [33]:
import requests
from bs4 import BeautifulSoup
import json
from kedro.io import DataCatalog

# Scrape web html
def _get_url_soup(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup


# Extract the season links and store IDs and names in dictionary/json AND SAVE AS A FILE SO IT CAN BE STORED IN THE DATASET
def _extract_season_ids(soup):
    select_element = soup.find("select", id="form1_selectedSeason")
    season_elements = select_element.find_all("option")
    season_ids = {}
    for element in season_elements:
        season_id = element['value']  # Extract the season ID
        season_name = element.text.strip()  # Get the season name
        season_ids[season_id] = season_name  # Store with ID as key
    return season_ids


def _save_to_json(dictionary: dict, catalog: DataCatalog, dataset: str):
    #Save json file so that it can be loaded in later 
    catalog.save(dictionary, dataset)

In [36]:
# Scrape the season IDs and names
soup = _get_url_soup("https://fulltime.thefa.com/index.html?league=847518011&selectedSeason=249912101&selectedDivision=272973748&selectedCompetition=0&selectedFixtureGroupKey=1_467585716")

season_ids = _extract_season_ids(soup) 

_save_to_json(season_ids, catalog, "season_ids") 

NameError: name 'catalog' is not defined

In [28]:
print(season_ids)

{'249912101': '2012-13', '548382721': '2013-14', '758918628': '2014-15', '608145920': '2015-16', '297987566': '2016-17', '882260688': '2017-18', '211813707': '2018-19', '983214482': '2019-20', '580529019': '2020-21', '739101976': '2021-22', '662359190': '2022-23', '995845630': '2023-24', '572282993': '2024-25'}


In [ ]:
from kedro.pipeline import Pipeline, node

# Define the pipeline
pipeline = Pipeline([
    node(
        func=_save_to_json,
        inputs=dict(
            season_ids="season_ids",
            catalog="catalog",
            dataset_name="params:season_ids_dataset"
        ),
        outputs=None,
        name="save_season_ids_node"
    )
])